# 🎯 Entrenamiento de Modelo YOLO para Detección de EPP

## 📋 Preparación Local (ANTES de usar Colab)

### Opción 1: Dataset Ya Organizado (Recomendado)
```powershell
# En tu PC (PowerShell)
cd API\models
python organize_dataset_for_yolo.py
Compress-Archive -Path "EPP_dataset" -DestinationPath "EPP_dataset.zip"
```
**Resultado**: `EPP_dataset.zip` (~3-5 GB)

### Opción 2: Dataset por Carpetas
```powershell
# Si ya generaste el dataset augmentado
Compress-Archive -Path "imagenes_epp_augmented" -DestinationPath "imagenes_epp_augmented.zip"
```
**Resultado**: `imagenes_epp_augmented.zip` (~3-5 GB)

---

## 🚀 Pasos en Google Colab

1. **Activar GPU**: Runtime → Change runtime type → T4 GPU
2. **Ejecutar celdas** en orden (1️⃣ → 1️⃣1️⃣)
3. **Subir ZIP** cuando se solicite (celda 3️⃣)
4. **Esperar** ~2-3 horas para entrenamiento completo

---

## 📝 Clases EPP (8)

0. **barbijo** - Mascarilla/Tapabocas
1. **botas** - Calzado de seguridad  
2. **camisa** - Camisa de trabajo
3. **casco** - Casco de seguridad
4. **chaleco** - Chaleco reflectivo
5. **guantes** - Guantes de protección
6. **lentes** - Gafas de seguridad
7. **pantalon** - Pantalón de trabajo

**Nota**: `epp_completo` se calcula programáticamente (no se entrena)

---

## ⚡ Consejos

- **GPU requerida**: El entrenamiento es muy lento en CPU
- **Tiempo estimado**: 2-3 horas con GPU T4 (50 épocas)
- **Dataset**: ~14,000 imágenes (8 clases)
- **Resultado**: Modelo `ppe_best.pt` para descargar

---

In [ ]:
# 1️⃣ Verificar GPU disponible
import torch
print(f"PyTorch versión: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU no detectada. Activa GPU en: Runtime → Change runtime type")

In [ ]:
# 2️⃣ Instalar Ultralytics (YOLOv8)
!pip install ultralytics -q
print("✅ Ultralytics instalado")

In [ ]:
# 3️⃣ Montar Google Drive y copiar dataset
from google.colab import drive
import os

# Montar Drive
print("📁 Montando Google Drive...")
drive.mount('/content/drive')

print("\n✅ Google Drive montado exitosamente")
print("\n🔍 Buscando EPP_dataset.zip en tu Drive...\n")

# Posibles ubicaciones del archivo
drive_paths = [
    '/content/drive/MyDrive/EPP_dataset.zip',
    '/content/drive/My Drive/EPP_dataset.zip',
]

zip_found = False
source_path = None

for path in drive_paths:
    if os.path.exists(path):
        source_path = path
        file_size = os.path.getsize(path) / (1024**3)  # GB
        print(f"✅ Encontrado: {path}")
        print(f"   Tamaño: {file_size:.2f} GB")
        zip_found = True
        break

if zip_found:
    print(f"\n📦 Copiando desde Drive a Colab (puede tardar 2-5 minutos)...")
    !cp "{source_path}" /content/EPP_dataset.zip
    
    # Verificar copia
    if os.path.exists('/content/EPP_dataset.zip'):
        print("✅ Archivo copiado exitosamente a /content/")
        print(f"   Tamaño: {os.path.getsize('/content/EPP_dataset.zip') / (1024**3):.2f} GB")
    else:
        print("❌ Error al copiar el archivo")
else:
    print("❌ No se encontró EPP_dataset.zip en tu Google Drive")
    print("\n📋 Instrucciones:")
    print("   1. Ve a https://drive.google.com")
    print("   2. Sube EPP_dataset.zip a 'Mi unidad' (raíz)")
    print("   3. Espera a que termine la subida")
    print("   4. Vuelve a ejecutar esta celda")
    print("\n🔍 Listando archivos en tu Drive:")
    !ls -lh "/content/drive/MyDrive/" | head -20


In [ ]:
# 4️⃣ Extraer dataset
import zipfile
import os

# Detectar qué archivo ZIP existe
if os.path.exists('EPP_dataset.zip'):
    zip_file = 'EPP_dataset.zip'
    extract_to = '.'
    print("📦 Extrayendo EPP_dataset.zip (ya organizado para YOLO)...")
elif os.path.exists('imagenes_epp_augmented.zip'):
    zip_file = 'imagenes_epp_augmented.zip'
    extract_to = '.'
    print("📦 Extrayendo imagenes_epp_augmented.zip...")
else:
    print("❌ No se encontró ningún archivo ZIP")
    print("   Sube EPP_dataset.zip o imagenes_epp_augmented.zip")
    raise FileNotFoundError("No se encontró archivo ZIP del dataset")

# Extraer
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset extraído")

# Verificar estructura
print("\n📁 Archivos extraídos:")
!ls -la

## 📁 Preparar Estructura del Dataset

Después de extraer el dataset, debes organizarlo en esta estructura:

```
EPP_dataset/
├── data.yaml
└── images/
    ├── train/         # 70% de las imágenes
    │   ├── barbijo_001.jpg
    │   ├── botas_002.jpg
    │   └── ...
    ├── val/           # 15% de las imágenes
    │   └── ...
    └── test/          # 15% de las imágenes (opcional)
        └── ...
```

**Nota:** Si usaste `generate_augmented_dataset.py`, tus imágenes están en carpetas por clase. Necesitas reorganizarlas en la estructura de YOLO.


In [ ]:
# 4️⃣.1 Organizar dataset (SOLO si subiste imagenes_epp_augmented.zip)
import os
import shutil
from pathlib import Path
import random

def organize_dataset_for_yolo(source_dir='imagenes_epp_augmented', output_dir='EPP_dataset'):
    """
    Organiza imágenes de carpetas por clase a estructura YOLO.
    
    Divide en:
    - 70% entrenamiento (train)
    - 15% validación (val)
    - 15% prueba (test)
    """
    
    # Clases (SIN epp_completo - se calcula programáticamente)
    classes = ['barbijo', 'botas', 'camisa', 'casco', 'chaleco', 
               'guantes', 'lentes', 'pantalon']
    
    # Crear estructura
    splits = ['train', 'val', 'test']
    for split in splits:
        os.makedirs(f'{output_dir}/images/{split}', exist_ok=True)
        os.makedirs(f'{output_dir}/labels/{split}', exist_ok=True)
    
    print("📁 Organizando dataset para YOLO (8 clases)...\n")
    
    total_images = 0
    stats = {'train': 0, 'val': 0, 'test': 0}
    
    for class_idx, class_name in enumerate(classes):
        class_dir = Path(source_dir) / class_name
        
        if not class_dir.exists():
            print(f"⚠️  Carpeta no encontrada: {class_name}")
            continue
        
        # Obtener todas las imágenes
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.png'))
        random.shuffle(images)
        
        # Calcular divisiones
        n_images = len(images)
        n_train = int(n_images * 0.70)
        n_val = int(n_images * 0.15)
        
        train_imgs = images[:n_train]
        val_imgs = images[n_train:n_train + n_val]
        test_imgs = images[n_train + n_val:]
        
        # Copiar imágenes y crear etiquetas
        for split, imgs in [('train', train_imgs), ('val', val_imgs), ('test', test_imgs)]:
            for img_path in imgs:
                # Copiar imagen
                dest_img = f"{output_dir}/images/{split}/{img_path.name}"
                shutil.copy2(img_path, dest_img)
                
                # Crear archivo de etiqueta
                # Para clasificación de imagen completa:
                label_path = f"{output_dir}/labels/{split}/{img_path.stem}.txt"
                with open(label_path, 'w') as f:
                    # Formato YOLO: class_id center_x center_y width height
                    # Imagen completa = 0.5 0.5 1.0 1.0
                    f.write(f"{class_idx} 0.5 0.5 1.0 1.0\n")
                
                stats[split] += 1
        
        total_images += n_images
        print(f"✓ {class_name:15s}: {n_train:4d} train + {n_val:4d} val + {len(test_imgs):4d} test = {n_images:4d} total")
    
    print(f"\n📊 Resumen:")
    print(f"  • Train:      {stats['train']:6d} imágenes")
    print(f"  • Validation: {stats['val']:6d} imágenes")
    print(f"  • Test:       {stats['test']:6d} imágenes")
    print(f"  • TOTAL:      {total_images:6d} imágenes")
    print(f"  • Clases:     {len(classes)} (sin epp_completo)")
    print(f"\n✅ Dataset organizado en: {output_dir}/")

# Verificar si necesitamos organizar
if os.path.exists('imagenes_epp_augmented'):
    print("🔍 Detectado: imagenes_epp_augmented/")
    print("   Organizando para YOLO (8 clases)...\n")
    organize_dataset_for_yolo('imagenes_epp_augmented', 'EPP_dataset')
elif os.path.exists('EPP_dataset/data.yaml'):
    print("✅ Dataset ya está organizado para YOLO")
    print("   Puedes continuar con el siguiente paso")
else:
    print("⚠️  No se encontró el dataset")
    print("   Verifica que subiste el archivo ZIP correcto")

In [ ]:
# 5️⃣ Verificar data.yaml
!cat EPP_dataset/data.yaml

In [ ]:
# 6️⃣ Configurar data.yaml con 8 clases EPP (sin epp_completo)
import yaml
import os

# Configuración del dataset
data_config = {
    'path': '/content/EPP_dataset',  # Ruta base del dataset
    'train': 'images/train',          # Carpeta de entrenamiento
    'val': 'images/val',              # Carpeta de validación
    'test': 'images/test',            # Carpeta de prueba (opcional)
    
    'nc': 8,  # Número de clases (sin epp_completo)
    'names': [
        'barbijo',      # 0
        'botas',        # 1
        'camisa',       # 2
        'casco',        # 3
        'chaleco',      # 4
        'guantes',      # 5
        'lentes',       # 6
        'pantalon'      # 7
    ]
}

# Guardar configuración
with open('EPP_dataset/data.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)

print("✅ data.yaml configurado con 8 clases EPP\n")
print(f"Clases detectables ({data_config['nc']}):")
for i, name in enumerate(data_config['names']):
    print(f"  {i}: {name}")

print("\n💡 Nota: 'epp_completo' se calculará programáticamente en el backend")
print("   EPP Completo = todas las 8 clases detectadas")

# Verificar contenido
print("\n📄 Contenido de data.yaml:")
with open('EPP_dataset/data.yaml', 'r') as f:
    print(f.read())


In [ ]:
# 7️⃣ Entrenar el modelo
from ultralytics import YOLO
import torch

print("🚀 Iniciando entrenamiento...\n")

# Detectar dispositivo disponible
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo: {'GPU' if device == 0 else 'CPU'}")

if device == 'cpu':
    print("⚠️  ADVERTENCIA: Entrenando en CPU (muy lento)")
    print("   Recomendación: Activa GPU en Runtime → Change runtime type → T4 GPU")
    input("Presiona ENTER para continuar de todos modos...")

# Cargar modelo base
model = YOLO('yolov8n.pt')

# Entrenar
results = model.train(
    data='EPP_dataset/data.yaml',
    epochs=50,              # 50 épocas
    imgsz=640,              # Tamaño de imagen
    batch=16 if device == 0 else 4,  # Batch más pequeño para CPU
    device=device,          # Usar GPU si está disponible, sino CPU
    project='runs/ppe',     # Carpeta de resultados
    name='train',           # Nombre del experimento
    patience=10,            # Early stopping
    save=True,              # Guardar modelo
    plots=True,             # Generar gráficas
    verbose=True
)

print("\n✅ Entrenamiento completado")

In [ ]:
# 8️⃣ Verificar resultados
!ls -lh runs/ppe/train/weights/

print("\n📊 Modelos generados:")
print("  - best.pt: Mejor modelo (menor loss)")
print("  - last.pt: Último checkpoint")

In [ ]:
# 9️⃣ Ver métricas de entrenamiento
from IPython.display import Image, display

print("📈 Curvas de entrenamiento:")
display(Image('runs/ppe/train/results.png'))

print("\n🔍 Matriz de confusión:")
display(Image('runs/ppe/train/confusion_matrix.png'))

In [ ]:
# 🔟 Probar el modelo entrenado
from PIL import Image
import matplotlib.pyplot as plt

# Cargar mejor modelo
trained_model = YOLO('runs/ppe/train/weights/best.pt')

# Buscar una imagen de prueba
import glob
test_images = glob.glob('EPP_dataset/train/images/*.jpg')[:3]

print(f"🧪 Probando con {len(test_images)} imágenes...\n")

for img_path in test_images:
    # Predecir
    results = trained_model(img_path)
    
    # Mostrar
    plt.figure(figsize=(10, 10))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.title(f"Detecciones en {img_path.split('/')[-1]}")
    plt.show()
    
    # Mostrar detecciones
    if len(results[0].boxes) > 0:
        print(f"  ✅ {len(results[0].boxes)} objeto(s) detectado(s)")
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            name = trained_model.names[cls]
            print(f"     - {name}: {conf:.2%}")
    else:
        print("  ℹ️  No se detectaron objetos")
    print()

In [ ]:
# 1️⃣1️⃣ Descargar modelo entrenado
from google.colab import files
import shutil

# Copiar y renombrar
shutil.copy('runs/ppe/train/weights/best.pt', 'ppe_best.pt')

print("📥 Descargando modelo entrenado...")
files.download('ppe_best.pt')

print("\n✅ Modelo descargado: ppe_best.pt")
print("\n📝 Siguiente paso:")
print("  1. Copia ppe_best.pt a: API/models/")
print("  2. Configura en .env:")
print("     MODEL_PATH=models/ppe_best.pt")
print("     USE_ROBOFLOW=False")
print("  3. Ejecuta: python main.py")

## 🎉 ¡Entrenamiento Completado!

### 📊 Resultados
- Modelo entrenado: `ppe_best.pt`
- Gráficas en: `runs/ppe/train/`

### 🔄 Próximos Pasos
1. Descarga `ppe_best.pt`
2. Colócalo en `API/models/`
3. Actualiza `.env`:
   ```
   MODEL_PATH=models/ppe_best.pt
   USE_ROBOFLOW=False
   ```
4. Inicia la API: `python main.py`

### 🔧 Si quieres mejorar el modelo
- Aumenta `epochs` (ej: 100-200)
- Usa modelo más grande: `yolov8s.pt` o `yolov8m.pt`
- Ajusta `batch` según memoria GPU
- Aplica más augmentation

### 📝 Clases detectadas (8)
0. **barbijo** - Mascarilla/Tapabocas
1. **botas** - Calzado de seguridad
2. **camisa** - Camisa de trabajo
3. **casco** - Casco de seguridad
4. **chaleco** - Chaleco reflectivo
5. **guantes** - Guantes de protección
6. **lentes** - Gafas de seguridad
7. **pantalon** - Pantalón de trabajo

**Nota**: `epp_completo` se calcula programáticamente cuando se detectan las 8 clases

### 💡 Tips para mejor precisión
- Usa el dataset augmentado (~14,000 imágenes)
- Entrena por más épocas (100+)
- Valida con imágenes reales no vistas
- Ajusta threshold de confianza en producción

### ⭐ Ventajas del nuevo enfoque
- ✅ **Más preciso**: 100% en cada clase vs 0% en epp_completo
- ✅ **Más flexible**: Ajusta qué constituye EPP completo sin re-entrenar
- ✅ **Mejor UX**: Usuario ve exactamente qué piezas faltan
- ✅ **Más confiable**: Basado en detecciones individuales verificadas
